# An array of sequences

*Example 2-4. Cartesian product using a list comprehension.*

In [4]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts 

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

## Generator Expressions
*Example 2-5. Initializing a tuple and an array from a generator expression*